### pip install & import

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
#import pandas_profiling
import datetime

In [2]:
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder

from sklearn.preprocessing import MinMaxScaler

from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestRegressor

# from xgboost import XGBRFRegressor
from sklearn.model_selection import cross_validate

In [ ]:
# pip install pycaret

In [ ]:
# from pycaret.regression import *

In [ ]:
# s = setup(movie_final_회귀,target='전국 관객수',train_size=0.8)

In [ ]:
# best = compare_models(sort='RMSE',n_select=5,fold=10)

### 최종데이터 불러오기

In [3]:
movie_final_회귀 = pd.read_csv('../3_merge_top300/movie_final_reg.csv',encoding='utf-8')
movie_final_분류 = pd.read_csv('../3_merge_top300/movie_final_cls.csv',encoding='utf-8')

## 원핫인코딩 

In [ ]:
# 원핫인코딩

# movie_labels = pd.get_dummies(movie_final,columns = ['국적','장르','등급','계절'])

In [ ]:
# 독립 변수와 종속 변수 나눔

# 독립 = movie_DL[['감독_흥행', '배급사_흥행', '국적_기타', '국적_독일', '국적_미국',
#        '국적_영국', '국적_일본', '국적_중국', '국적_프랑스', '국적_한국', '장르_SF', '장르_가족', '장르_공연',
#        '장르_공포(호러)', '장르_기타', '장르_다큐멘터리', '장르_드라마', '장르_멜로/로맨스', '장르_뮤지컬',
#        '장르_미스터리', '장르_범죄', '장르_사극', '장르_서부극(웨스턴)', '장르_성인물(에로)', '장르_스릴러',
#        '장르_애니메이션', '장르_액션', '장르_어드벤처', '장르_전쟁', '장르_코미디', '장르_판타지',
#        '등급_12세관람가', '등급_15세관람가', '등급_전체관람가', '등급_청소년관람불가', '계절_가을', '계절_겨울',
#        '계절_봄', '계절_여름']] 
# 종속 = movie_DL[['전국 관객수']] 

# 스케일링

In [4]:
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer

# 객체생성
robust_scaler = RobustScaler()
mMscaler = MinMaxScaler()
norm = Normalizer()

# *** 만약 특정 열의 스케일링을 하고 싶은 경우 ***
A_n = robust_scaler.fit_transform(movie_final_회귀['전국 스크린수'].values.reshape(-1,1))
movie_final_회귀.insert(0, '전국 스크린수_1', A_n)
movie_final_회귀.drop(['전국 스크린수'], axis=1, inplace=True)

A_n1 = robust_scaler.fit_transform(movie_final_회귀['총 관객수'].values.reshape(-1,1))
movie_final_회귀.insert(0, '총 관객수_1', A_n1)
movie_final_회귀.drop(['총 관객수'], axis=1, inplace=True)

A_n2 = robust_scaler.fit_transform(movie_final_회귀['평점'].values.reshape(-1,1))
movie_final_회귀.insert(0, '평점_1', A_n1)
movie_final_회귀.drop(['평점'], axis=1, inplace=True)

## 라벨링

In [5]:
from sklearn.preprocessing import LabelEncoder

le_form = LabelEncoder()
le_nation = LabelEncoder()
le_genre = LabelEncoder()
le_rating = LabelEncoder()
le_class = LabelEncoder()
le_season = LabelEncoder()

movie_labels = movie_final_회귀.copy()
movie_labels['영화형태'] = le_form.fit_transform(movie_final_회귀['영화형태'])
movie_labels['국적'] = le_nation.fit_transform(movie_final_회귀['국적'])
movie_labels['장르'] = le_genre.fit_transform(movie_final_회귀['장르'])
movie_labels['등급'] = le_rating.fit_transform(movie_final_회귀['등급'])
movie_labels['영화구분'] = le_class.fit_transform(movie_final_회귀['영화구분'])
movie_labels['계절'] = le_season.fit_transform(movie_final_회귀['계절'])
movie_labels.drop(['Unnamed: 0'],axis=1,inplace=True)
movie_labels.head(3)


,평점_1,총 관객수_1,전국 스크린수_1,순번,영화명,감독,배급사,개봉일,영화형태,국적,...,등급,영화구분,연도,월,계절,코로나,출연,감독_흥행,배급사_흥행,주연배우_흥행
0,0.018839,0.018839,1.959732,2575,미니특공대X,이영준,삼지애니메이션,2018-03-22,2,7,...,2,1,2018,3,2,0,"['엄상현', '전태열', '이소영', '신용우']",0,0,0
1,-0.854008,-0.854008,-0.033557,2578,알파독,닉 카사베츠,싸이더스,2009-01-29,2,2,...,3,1,2009,1,1,0,"['에밀 허쉬', '저스틴 팀버레이크', '벤 포스터', '안톤 옐친', '아만다 ...",0,6,6
2,-1.463612,-1.463612,3.348993,2583,엘비스,바즈 루어만,워너브러더스 코리아,2022-07-13,2,2,...,1,1,2022,7,3,1,"['오스틴 버틀러', '톰 행크스']",0,10,5


### 데이터분배
1) movie_labels_etc : train과 val data   
2) movie_labels_2023 : test data

In [50]:
movie_labels_2023  = movie_labels.sample(frac=0.2)
movie_labels_etc = movie_labels.sample(frac=0.8)

In [6]:
movie_labels_2023 = movie_labels[movie_labels['연도']==2023]
movie_labels_etc = movie_labels[movie_labels['연도']!=2023]

### 변수 정리   
1. 사용하지 않는 컬럼 제거
2. 독립변수와 종속변수 설정

In [51]:
# 사용하지 않는 컬럼 제거

not_used = ['영화명', '감독', '배급사', '영화형태', '개봉일', '영화구분', '월']

movie_DL = movie_labels_etc.drop(not_used, axis=1) # 2003~2022 데이터 main
movie_DL2 = movie_labels_2023.drop(not_used, axis=1)# 2023년 test


In [ ]:
# 국적 => 국내 vs 해외
# 관객비율.. 2023년 데이터 총 관객수 몰라....
# 비율 말고 총 관객수를 쓰자
# 총관객수 defqult를 2022년으로 해서 2023년 총관객수 채우면 어때?

In [52]:
# 독립 변수와 종속 변수 나눔

종속 = movie_DL[['전국 관객수']].copy()
종속['전국 관객수'] = 종속['전국 관객수'] / 1000 

종속_2023 = movie_DL2[['전국 관객수']].copy()
종속_2023['전국 관객수'] = 종속_2023['전국 관객수'] / 1000

In [53]:
독립 = movie_DL[['장르', '등급', '감독_흥행','총 관객수_1','평점_1',
               '배급사_흥행','주연배우_흥행','계절', '전국 스크린수_1','코로나']].copy()

독립_2023 = movie_DL2[['장르', '등급', '감독_흥행','총 관객수_1','평점_1',
               '배급사_흥행','주연배우_흥행','계절', '전국 스크린수_1','코로나']].copy()

In [ ]:
# 독립 = movie_DL[['전국 스크린수','총 관객수','평점', '감독_흥행', '배급사_흥행', '주연배우_흥행', '국적_기타',
#        '국적_독일', '국적_미국', '국적_영국', '국적_일본', '국적_중국', '국적_프랑스', '국적_한국', '장르_SF',
#        '장르_가족', '장르_공연', '장르_공포(호러)', '장르_기타', '장르_다큐멘터리', '장르_드라마',
#        '장르_멜로/로맨스', '장르_뮤지컬', '장르_미스터리', '장르_범죄', '장르_사극', '장르_성인물(에로)',
#        '장르_스릴러', '장르_애니메이션', '장르_액션', '장르_어드벤처', '장르_전쟁', '장르_코미디', '장르_판타지',
#        '등급_12세관람가', '등급_15세관람가', '등급_전체관람가', '등급_청소년관람불가', '계절_가을', '계절_겨울',
#        '계절_봄', '계절_여름', '코로나_0', '코로나_1']].copy()

# 독립_2023 = movie_DL2[['전국 스크린수','총 관객수','평점', '감독_흥행', '배급사_흥행', '주연배우_흥행', '국적_기타',
#        '국적_독일', '국적_미국', '국적_영국', '국적_일본', '국적_중국', '국적_프랑스', '국적_한국', '장르_SF',
#        '장르_가족', '장르_공연', '장르_공포(호러)', '장르_기타', '장르_다큐멘터리', '장르_드라마',
#        '장르_멜로/로맨스', '장르_뮤지컬', '장르_미스터리', '장르_범죄', '장르_사극', '장르_성인물(에로)',
#        '장르_스릴러', '장르_애니메이션', '장르_액션', '장르_어드벤처', '장르_전쟁', '장르_코미디', '장르_판타지',
#        '등급_12세관람가', '등급_15세관람가', '등급_전체관람가', '등급_청소년관람불가', '계절_가을', '계절_겨울',
#        '계절_봄', '계절_여름', '코로나_0', '코로나_1']].copy()

## train, test 데이터 나누기

In [54]:
# train, test 데이터 나눔

from sklearn.model_selection import train_test_split
train_input, test_input, train_target,test_target = train_test_split(독립,종속,test_size=0.3,random_state=42)

## 스케일링... 
...어디서 할까...<br>
라벨링 때문에 바로 스케일링을 따로 해줘야함 (전국스크린수, 전국매출액)

In [ ]:
# # 값 스케일링

# from sklearn.preprocessing import StandardScaler
# ss = StandardScaler()
# ss.fit(train_input)
# train_scaled = ss.transform(train_input)
# test_scaled = ss.transform(test_input)

# 딥러닝 


In [223]:
# 딥러닝 층 여기가 중요

X = tf.keras.layers.Input(shape=[10])

H = tf.keras.layers.Dense(256)(X)
H = tf.keras.layers.BatchNormalization(epsilon=1e-3)(H)
H = tf.keras.layers.Activation(tf.nn.relu)(H)

H = tf.keras.layers.Dense(256)(H)
H = tf.keras.layers.BatchNormalization(epsilon=1e-3)(H)
H = tf.keras.layers.Activation(tf.nn.relu)(H)

H = tf.keras.layers.Dense(256)(H)
H = tf.keras.layers.BatchNormalization(epsilon=1e-4)(H)
H = tf.keras.layers.Activation(tf.nn.relu)(H)
dropout = tf.keras.layers.Dropout(rate=0.3)(H)

Y = tf.keras.layers.Dense(1,activation='relu')(dropout)
model = tf.keras.models.Model(X,Y)
sgd = tf.keras.optimizers.Adam(learning_rate=0.001,beta_1=0.9,beta_2=0.99)
# sgd = tf.keras.optimizers.SGD(learning_rate=0.0001,momentum=0.9)
# sgd = tf.keras.optimizers.Adamax(learning_rate=0.001,beta_1=0.9,beta_2=0.99)
model.compile(optimizer=sgd,loss='mae')

In [224]:
# 훈련

history = model.fit(train_input,train_target,batch_size=128,epochs=100,validation_data=(독립_2023,종속_2023))

Epoch 1/100
7/7 [==============================] - 1s 29ms/step - loss: 31.3295 - val_loss: 39.0239
Epoch 2/100
7/7 [==============================] - 0s 10ms/step - loss: 29.2684 - val_loss: 39.0197
Epoch 3/100
7/7 [==============================] - 0s 10ms/step - loss: 28.4475 - val_loss: 39.0077
Epoch 4/100
7/7 [==============================] - 0s 10ms/step - loss: 27.9382 - val_loss: 38.9841
Epoch 5/100
7/7 [==============================] - 0s 10ms/step - loss: 27.5469 - val_loss: 38.9541
Epoch 6/100
7/7 [==============================] - 0s 10ms/step - loss: 27.2889 - val_loss: 38.8777
Epoch 7/100
7/7 [==============================] - 0s 9ms/step - loss: 27.1062 - val_loss: 38.7519
Epoch 8/100
7/7 [==============================] - 0s 9ms/step - loss: 26.8757 - val_loss: 38.5784
Epoch 9/100
7/7 [==============================] - 0s 9ms/step - loss: 26.7384 - val_loss: 38.2494
Epoch 10/100
7/7 [==============================] - 0s 9ms/step - loss: 26.5748 - val_loss: 37.3765
Epo

In [222]:
# 확인용도

# 오차율 몇 %로...?
pred = model.predict(독립_2023).round(0)
# 여기랑 비슷한지 보기
종속_2023['pred'] = pred
종속_2023['오차율'] = (abs(종속_2023['전국 관객수'] - 종속_2023['pred'])) / 종속_2023['전국 관객수'] * 100
len(종속_2023.loc[종속_2023['오차율']<20])

9/9 [==============================] - 0s 3ms/step


120

In [209]:
종속_2023

,전국 관객수,pred,오차율
1071,17.040,20.0,17.370892
1107,15.986,16.0,0.087577
906,22.097,69.0,212.259583
1216,13.314,29.0,117.815833
749,27.569,16.0,41.963800
...,...,...,...
1110,15.942,17.0,6.636558
644,32.752,13.0,60.307767
563,38.149,27.0,29.224881
969,19.623,28.0,42.689701


In [116]:
# 확인용도
model.predict(train_input[:5])

1/1 [==============================] - 0s 23ms/step


array([[29.363327],
       [15.453041],
       [12.37143 ],
       [74.71347 ],
       [80.61479 ]], dtype=float32)

In [210]:
# 여기랑 비슷한지 보기
train_target[:5]

,전국 관객수,pred,오차율
735,28.082,29.0,3.268998
1174,14.374,15.0,4.355086
1195,13.853,12.0,13.376164
104,83.474,75.0,10.151664
24,96.305,81.0,15.892217


### test data 확인용도

In [788]:
# model = tf.keras.models.load_model('./low_level.h5')